In [ ]:
# a notebook for working with LLMs locally using ollama models and langchain for prompt construction
# this code analyzes data from r/antiwork to explore collective/individual action frames

In [ ]:
# notebook setup, sometimes required
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# install required packages
!pip install langchain-ollama
!pip install ollama

In [ ]:
# import packages
import threading
import subprocess
import time
import pandas as pd
import ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama.llms import OllamaLLM
from IPython.display import Markdown

In [ ]:
# a function to start the ollama server for running LLMs locally
def run_ollama_serve():
  subprocess.Popen(['ollama', 'serve'])
thread = threading.Thread(target=run_ollama_serve)

thread.start()
time.sleep(15)

In [ ]:
# download the models (full list of options here: https://ollama.com/library)
ollama.pull("llama3.2")
ollama.pull("mistral")

In [ ]:
# for importing data from google drive; otherwise, ignore this block
from google.colab import drive
drive.mount('/content/drive/')

gdrive_path = "/content/drive" + "/My Drive" + "/folder_name"
df = pd.read_csv(gdrive_path + 'data.csv')

In [ ]:
# set the model (note: must be downloaded first in ollama.pull step)
model = OllamaLLM(model="llama3.2")

In [ ]:
# set the system message, which assigns the model an "identity"
system_message = SystemMessage(
    content= "You are Doreen, and you classify posts from the subreddit Antiwork as if you are the moderator of the community. Antiwork users discuss issues related to capitalism and work."
)


In [ ]:
# function to query the LLM, including an input prompt and text data in "messages" format, and save the response
def get_model_response(text):
    prompt = f"""
    Doreen, please state whether this post contains an individual call to action,
    a collective call to action, both an individual and a collective call to action,
    or no call to action. Explain your reasoning and what influenced your decision: {text}
    """

  messages = [
      system_message,
      HumanMessage(content=prompt)
  ]

  response = model.invoke(messages)
  return response

In [ ]:
# create a new column in the dataframe to save the output from llama based on text input from the 'selftext' column in the dataframe, which contains full texts of r/Antiwork posts
df['llama_response'] = df['selftext'].apply(get_model_response)

In [ ]:
# print the first 5 rows of the new column
df['llama_response'].head()

In [ ]:
# repeat the analysis with mistral, starting with setting the model parameter
model = OllamaLLM(model="mistral")

In [ ]:
# create a new column for the output from mistral based on the text column in the dataframe
df['mistral_response'] = df['selftext'].apply(get_model_response)

In [ ]:
# print the first 5 rows of the full dataset
df.head()

In [ ]:
# save the dataframe (this example saves to the mounted google drive location)
df.to_csv(gdrive_path + '/llm_results.csv')